In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

vistas.head()

,idaviso,idpostulante,cantidad,total_vistas_postulante
0,1111780242,YjVJQ6Z,0,0
1,1112263876,BmVpYoR,0,0
2,1112327963,wVkBzZd,0,0
3,1112318643,OqmP9pv,0,0
4,1111903673,DrpbXDP,0,0


In [7]:
#seteo el peso como cantidad de vistas del postulante al anuncio/total de vistas del postulante
vistas["cantidad"] = vistas["cantidad"]/vistas["total_vistas"]
vistas = vistas.drop("total_vistas",axis=1)
vistas.head()


,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,1738407,BolNL,0.035714
2,1795995,BolNL,0.071429
3,1804592,BolNL,0.035714
4,1804723,BolNL,0.035714


In [8]:
postulaciones2= pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones2=pd.merge(postulaciones2,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones2=pd.merge(postulaciones2,avisos,on='idaviso')

postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,graduados,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social
0,1112430536,EexaOz,0,MASC,38.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,False
1,1112430536,owaWw0Z,0,FEM,20.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False
2,1112430536,PmGPRd1,0,MASC,19.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False
3,1112430536,bO4q324,0,MASC,25.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False
4,1112430536,MVd5dlr,0,MASC,30.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False


In [9]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,graduados,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,True,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,False,False,False,False,True,True,False,False,True,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,True,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,False,False,False,False,True,True,False,False,True,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,True,False


In [10]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

12793168
12787587


In [11]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social,cantidad
754363,1112437803,1QkNZwR,1,MASC,28.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,True,True,False,NaN
483797,1112417178,eaAEm8,1,FEM,36.0,-,En Curso,-,Graduado,Graduado,...,True,False,False,False,False,False,True,False,False,0.033333
1808481,1112308402,Z9EXdY,1,MASC,26.0,-,-,-,-,-,...,True,False,False,False,False,False,True,False,False,NaN
6744824,1112501072,EGYGO2,0,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
9068106,1112278938,Nz09KxJ,0,FEM,31.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
782057,1111739153,8MrV2mL,1,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
10968222,1112432317,2zQB236,0,FEM,28.0,-,Graduado,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,NaN
5409972,1112251166,EMXA20,1,FEM,37.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
2316937,1112282696,ak53mGm,1,FEM,21.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,NaN
4652281,1112356396,W9PqZZ9,1,FEM,31.0,-,-,-,-,-,...,False,False,False,False,True,True,False,False,False,NaN


In [12]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
postulaciones2=0

Train:  11508828 Test:  1278759


In [13]:
import gc
gc.collect()

39

In [14]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=300,max_depth=9,n_estimators=1000,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [15]:
score=c.score(x_test,y_test)*100
print(score)

80.66257989191082


C:\Users\Pocho\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
joblib.dump(c, "Data/fiuba_entrenamiento/pocho/modelos/LGBMV9.pkl")

['Data/fiuba_entrenamiento/pocho/modelos/LGBMV9.pkl']

In [17]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social,cantidad
65111,11741,1111969407,EppvV4,MASC,43.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
93826,93043,1112477736,5Z40vM,FEM,42.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.000000
72683,40512,1112419243,JBXR255,MASC,46.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.115385
86381,37910,1112414517,W9VaYvX,MASC,23.0,-,-,-,-,Graduado,...,True,False,False,False,False,True,False,False,True,0.000000
60371,46143,1112429742,A360DW1,MASC,22.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.018182
47992,65923,1112458873,NzPodq4,MASC,21.0,-,-,-,-,Graduado,...,False,True,False,True,False,True,False,False,False,0.000000
70368,15847,1112146974,5avDN,MASC,44.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,True,False,False,0.000000
45279,12090,1111981382,a8OqpD,FEM,35.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
61842,9567,1111853014,eppx9E,FEM,32.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,False,False,False,0.000000
37349,59174,1112453552,jkjpQA0,MASC,25.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.054545


In [18]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.76575421, 0.23424579],
       [0.73976306, 0.26023694],
       [0.76868314, 0.23131686],
       ...,
       [0.92595848, 0.07404152],
       [0.86099302, 0.13900698],
       [0.96563712, 0.03436288]])

In [19]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [20]:
prediccion['sepostulo']=y_final2

In [21]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.234246
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.260237
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.231317
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.073214
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.055882


In [22]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.234246
1,1,0.260237
2,2,0.231317
3,3,0.073214
4,4,0.055882


In [23]:
len(prediccion)

100000

In [24]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/prediccion9.csv",index=False)
prediccion.sepostulo.mean()

0.5674602479567669